In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [ ]:
def my_func(a, b, n_samples, noise_var):
    x = np.linspace(a, b, n_samples)
    y = 2**np.cos(x**2) + np.random.normal(0, noise_var, n_samples)
    return x.reshape(-1, 1), y.reshape(-1, 1)

Try with noise_var = 0, 0.1, 0.5

In [ ]:
X_np, y_np = my_func(-np.pi, np.pi, 30, 0.0)

#X_np, y_np = my_func(-np.pi, np.pi, 500, 0.1)
#X_np, y_np = my_func(-np.pi, np.pi, 500, 0.5)

plt.plot(X_np, y_np, 'ro')
plt.show()

In [ ]:
X = torch.from_numpy(X_np.astype(np.float32))
y = torch.from_numpy(y_np.astype(np.float32))
y = y.view(y.shape[0], 1)

n_samples, n_features = X.shape

input_size = n_features
output_size = 1

Try with the commented lines

In [ ]:
class DNN(nn.Module):
    def __init__(self, n):
        super(DNN, self).__init__()
        
        # Número de capas (L = número de elementos en n - 1)
        self.L = len(n) - 1
        
        # Función de activación: guardamos la clase, instanciamos por capa
        #actcls = nn.Sigmoid
        #actcls = nn.ReLU
        actcls = nn.Tanh
        
        layers = list()
        # Construye las capas ocultas con su función de activación
        for i in range(self.L - 1):
            layers.append((f'l_{i+1}', nn.Linear(n[i], n[i+1])))  # Capa lineal
            layers.append((f'actfunc_{i+1}', actcls()))          # Activación (nueva instancia por capa)
        
        # Última capa (sin activación)
        layers.append((f'l_{self.L}', nn.Linear(n[-2], n[-1])))
        self.layers = nn.Sequential(OrderedDict(layers))  # Secuencia ordenada de capas
        
    def forward(self, x):
        # Propagación hacia adelante: aplica todas las capas secuencialmente
        return self.layers(x)
    

#model = DNN(n = [input_size, output_size])
model = DNN(n = [input_size, 20, output_size])
#model = DNN(n = [input_size, 20, 20, 20, 20, 20, output_size])
#model = DNN(n = [input_size, 200, output_size])

In [ ]:
y_hat = model(X)

plt.plot(X_np, y_np, 'ro')
plt.plot(X_np, y_hat.detach().numpy(), 'b')
plt.title('Initial Model - Neural Network')
plt.show()
plt.close()

In [ ]:
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
criterion = nn.MSELoss()

In [ ]:
MaxIters = 100000
for epoch in range(MaxIters):
    
    #Forward
    optimizer.zero_grad()
    y_hat = model(X)
    Err = criterion(y_hat, y)
    
    #Backward
    Err.backward()
    
    #Updating 
    optimizer.step()
    
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{MaxIters}], Loss: {Err.item():.4f}')

In [ ]:
plt.plot(X_np, y_np, 'ro')
plt.plot(X_np, y_hat.detach().numpy(), 'b')
plt.title('Trained Model - Neural Network')
plt.show()

In [ ]:
def Accuracy(y, y_hat):
    mse = torch.mean((y - y_hat)**2)
    return 1 / (1 + mse)

# Imprime metricas
print(f'Final Loss: {Err.item():.4f}')
print(f'Final Accuracy: {Accuracy(y, y_hat):.4f}')
print(f'RMSE: {torch.sqrt(torch.mean((y - y_hat)**2)).item():.4f}')
